In [2]:
import os
import cv2
import numpy as np

# Ruta al directorio de datos
data_dir = 'C:\\datasets\\archive\\train'

# Lista de etiquetas de emociones
emotions = ['happy', 'sad', 'surprise', 'angry', 'disgust', 'fear', 'neutral']

# Dimensiones deseadas de las imágenes
img_size = (200, 200)

# Inicializar listas para las imágenes y etiquetas
faces = []
labels = []

# Función para cargar imágenes en lotes
def load_images_batch(emotion_dir, emotion_label, batch_size=100):
    batch_faces = []
    batch_labels = []
    img_files = os.listdir(emotion_dir)[:1000] ## Limitar a 1000 imgs por cat
    
    for i, img_name in enumerate(img_files):
        img_path = os.path.join(emotion_dir, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, img_size)
            batch_faces.append(img)
            batch_labels.append(emotion_label)
        
        # Procesar en lotes
        if (i + 1) % batch_size == 0 or (i + 1) == len(img_files):
            yield np.array(batch_faces), np.array(batch_labels)
            batch_faces, batch_labels = [], []

# Cargar las imágenes y etiquetas en lotes
for emotion in emotions:
    emotion_dir = os.path.join(data_dir, emotion)
    if not os.path.exists(emotion_dir):
        print(f"La carpeta {emotion_dir} no existe, revisa la ruta.")
        continue

    for batch_faces, batch_labels in load_images_batch(emotion_dir, emotions.index(emotion)):
        faces.extend(batch_faces)
        labels.extend(batch_labels)

# Convertir las listas a arrays de numpy
faces = np.array(faces)
labels = np.array(labels)

# Asegurarse de que las imágenes tengan el formato correcto
faces = faces.reshape(faces.shape[0], img_size[0], img_size[1], 1)
faces = faces.astype('float32') / 255.0

print(f"Cargadas {faces.shape[0]} imágenes con dimensiones {faces.shape[1:]}")

# dividir en entrenamiento y prueba
from sklearn.model_selection import train_test_split

faces_train, faces_test, labels_train, labels_test = train_test_split(faces, labels, test_size=0.2, random_state=42)

print(f"Conjunto de entrenamiento: {faces_train.shape[0]} imágenes")
print(f"Conjunto de prueba: {faces_test.shape[0]} imágenes")


# Crear el modelo de Eigenfaces
model = cv2.face.EigenFaceRecognizer_create()

# Entrenar el modelo
model.train(faces_train, labels_train)

# Guardar el modelo entrenado
model.save('modelo_eigenfaces.xml')


# Cargar el modelo entrenado
model = cv2.face.EigenFaceRecognizer_create()
model.read('modelo_eigenfaces.xml')

# Iniciar la captura de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (200, 200))  # Cambiar el tamaño a 200x200
        
        # Predecir la expresión facial
        label, confidence = model.predict(roi_gray)
        emotion = emotions[label]
        
        # Dibujar un rectángulo alrededor de la cara y mostrar la emoción
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Mostrar el video
    cv2.imshow('Detección de Expresiones Faciales', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Cargadas 6436 imágenes con dimensiones (200, 200, 1)
Conjunto de entrenamiento: 5148 imágenes
Conjunto de prueba: 1288 imágenes


In [3]:

import cv2


# Cargar el modelo entrenado
model = cv2.face.EigenFaceRecognizer_create()
model.read('modelo_eigenfaces.xml')

# Iniciar la captura de video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml').detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (200, 200))  # Cambiar el tamaño a 200x200
        
        # Predecir la expresión facial
        label, confidence = model.predict(roi_gray)
        emotion = emotions[label]
        
        # Dibujar un rectángulo alrededor de la cara y mostrar la emoción
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    
    # Mostrar el video
    cv2.imshow('Detección de Expresiones Faciales', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

